# PS1: League Winner Prediction - Rigorous Retraining

## ⚠️ CRITICAL CORRECTION: ADDRESSING DATA LEAKAGE

My previous analysis was flawed and produced misleading "perfect" scores. This was due to **severe data leakage**, where features that are derived from the target variable were used for training. This is a major methodological error.

**The primary source of leakage was using `points_per_game` as a feature, which is directly calculated from `target_total_points`.**

This notebook has been completely rewritten to follow proper machine learning best practices.

### Corrected Workflow:
1.  **Isolate Target:** The target variable (`target_champion`) and any derivatives are strictly used for evaluation, not as features.
2.  **Eliminate Leaky Features:** All columns that would not be available *before* a season starts are removed from the feature set.
3.  **Temporal Validation:** The data is split by season to simulate a real-world prediction scenario (training on past seasons to predict a future one).
4.  **Realistic Evaluation:** Performance is measured on a hold-out test set of the most recent season(s), providing an honest assessment of the model's capabilities.

### Prediction Goal
- **Predict the league champion (`target_champion` = 1) for a given season.**

## 1. Import Libraries and Setup

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import json
from datetime import datetime

from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# Import XGBoost and LightGBM if available
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False

try:
    import lightgbm as lgb
    LGB_AVAILABLE = False # Disabled for now to ensure robust, non-leaky models first
except ImportError:
    LGB_AVAILABLE = False

# --- Configuration ---
DATA_URL = '../data/final/data_final_points_tally.csv'
TARGET_COL = 'target_champion'
MODELS_DIR = Path('models')
MODELS_DIR.mkdir(exist_ok=True)

# --- Load Data ---
print("Loading dataset...")
df = pd.read_csv(DATA_URL)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# --- CRITICAL: Define Leaky and Unusable Columns ---
# These columns contain information about the outcome of the season, so they cannot be used as features.
LEAKAGE_COLS = [
    'points_per_game',          # Directly calculated from target_total_points
    'target_total_points',      # A target variable itself
    'target_league_position',   # Another target variable
    'target_champion'           # The primary target for this problem statement
]

# Define feature columns - exclude leakage and identifiers
feature_cols = [
    col for col in df.columns
    if col not in LEAKAGE_COLS and col not in ['season_encoded', 'team_encoded']
]

print("\n--- Feature and Target Definition ---")
print(f"Target column: '{TARGET_COL}'")
print(f"Feature columns ({len(feature_cols)}): {feature_cols}")
print(f"Leakage columns removed: {LEAKAGE_COLS}")

# --- Prepare Data for Temporal Validation ---
# Sort data by season to ensure correct temporal order
df_sorted = df.sort_values('season_encoded').reset_index(drop=True)

X = df_sorted[feature_cols]
y = df_sorted[TARGET_COL]

print(f"\nFeatures (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print(f"Champion distribution:\n{y.value_counts(normalize=True)}")

Loading dataset...
Dataset shape: (180, 10)
Columns: ['matches_played', 'points_per_game', 'goals_scored', 'goals_conceded', 'goal_difference', 'target_total_points', 'target_league_position', 'target_champion', 'season_encoded', 'team_encoded']

--- Feature and Target Definition ---
Target column: 'target_champion'
Feature columns (4): ['matches_played', 'goals_scored', 'goals_conceded', 'goal_difference']
Leakage columns removed: ['points_per_game', 'target_total_points', 'target_league_position', 'target_champion']

Features (X) shape: (180, 4)
Target (y) shape: (180,)
Champion distribution:
target_champion
0    0.95
1    0.05
Name: proportion, dtype: float64


## 2. Load and Explore Data

In [18]:
# --- Temporal Train-Test Split ---
# We must train on past data and test on future data.

seasons = df_sorted['season_encoded'].unique()
print(f"Available seasons: {seasons}")

# Use the last season for testing, and all prior seasons for training
test_season = seasons[-1]
train_seasons = seasons[:-1]

train_mask = df_sorted['season_encoded'].isin(train_seasons)
test_mask = df_sorted['season_encoded'] == test_season

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

print("\n--- Temporal Split Details ---")
print(f"Training seasons: {train_seasons}")
print(f"Test season: {test_season}")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Champions in training set: {y_train.sum()}")
print(f"Champions in test set: {y_test.sum()}")

Available seasons: [0 1 2 3 4 5 6 7 8]

--- Temporal Split Details ---
Training seasons: [0 1 2 3 4 5 6 7]
Test season: 8
Training set size: 160 samples
Test set size: 20 samples
Champions in training set: 8
Champions in test set: 1


## 3. Prepare Features and Target

In [19]:
# --- Model Training Pipeline with Temporal Cross-Validation ---

# Define a robust pipeline
def create_pipeline(model):
    return Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('model', model)
    ])

# Define models to train
models = {
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42, class_weight='balanced')
}
if XGB_AVAILABLE:
    models['XGBoost'] = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Define hyperparameter search space
param_grids = {
    'GradientBoosting': {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.05, 0.1],
        'model__max_depth': [3, 5]
    },
    'RandomForest': {
        'model__n_estimators': [100, 200],
        'model__max_depth': [5, 10, None],
        'model__min_samples_leaf': [1, 2, 4]
    },
    'XGBoost': {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.05, 0.1],
        'model__max_depth': [3, 5],
        'model__scale_pos_weight': [len(y_train[y_train==0]) / len(y_train[y_train==1])]
    }
}

# Use TimeSeriesSplit for cross-validation to respect temporal order
n_splits = min(len(train_seasons) - 1, 5) # Cannot have more splits than seasons-1
tscv = TimeSeriesSplit(n_splits=n_splits)

print(f"\n--- Training Models with Temporal CV ({n_splits} splits) ---")

results = {}
for name, model in models.items():
    print(f"Training {name}...")
    pipeline = create_pipeline(model)
    
    # Use RandomizedSearchCV for efficiency
    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grids[name],
        n_iter=10,
        cv=tscv,
        scoring='f1_macro',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    search.fit(X_train, y_train)
    
    # Evaluate on the hold-out test set
    y_pred = search.predict(X_test)
    y_proba = search.predict_proba(X_test)[:, 1]
    
    results[name] = {
        'model': search.best_estimator_,
        'best_params': search.best_params_,
        'accuracy': accuracy_score(y_test, y_pred),
        'f1_macro': f1_score(y_test, y_pred, average='macro'),
        'roc_auc': roc_auc_score(y_test, y_proba),
        'report': classification_report(y_test, y_pred, zero_division=0),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }
    print(f"  Done. Best F1 (macro) on CV: {search.best_score_:.4f}")

# --- Display Results ---
print("\n--- Model Performance on Hold-out Test Set ---")
for name, res in results.items():
    print(f"\n----- {name} -----")
    print(f"  Accuracy: {res['accuracy']:.4f}")
    print(f"  F1 (Macro): {res['f1_macro']:.4f}")
    print(f"  ROC AUC: {res['roc_auc']:.4f}")
    print("  Classification Report:")
    print(res['report'])
    print("  Confusion Matrix:")
    print(res['confusion_matrix'])


--- Training Models with Temporal CV (5 splits) ---
Training GradientBoosting...
  Done. Best F1 (macro) on CV: 0.7365
Training RandomForest...
  Done. Best F1 (macro) on CV: 0.7365
Training RandomForest...
  Done. Best F1 (macro) on CV: 0.8186
Training XGBoost...
  Done. Best F1 (macro) on CV: 0.8186
Training XGBoost...
  Done. Best F1 (macro) on CV: 0.7367

--- Model Performance on Hold-out Test Set ---

----- GradientBoosting -----
  Accuracy: 0.9500
  F1 (Macro): 0.4872
  ROC AUC: 0.9474
  Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        19
           1       0.00      0.00      0.00         1

    accuracy                           0.95        20
   macro avg       0.47      0.50      0.49        20
weighted avg       0.90      0.95      0.93        20

  Confusion Matrix:
[[19  0]
 [ 1  0]]

----- RandomForest -----
  Accuracy: 0.9500
  F1 (Macro): 0.4872
  ROC AUC: 0.9474
  Classification Report:
  

## 4. Define Pipeline and Hyperparameters

## 5. Train Models

In [ ]:
# --- Save the Best Model ---
# Select the best model based on F1-score on the test set
best_model_name = max(results, key=lambda name: results[name]['f1_macro'])
best_model_data = results[best_model_name]
best_model = best_model_data['model']

print(f"\n--- Saving Best Model: {best_model_name} ---")
print(f"Test F1 (Macro): {best_model_data['f1_macro']:.4f}")

# Clean up old model files
for old_file in MODELS_DIR.glob('ps1_league_winner_*'):
    old_file.unlink()

# Save model and metadata
model_filename = MODELS_DIR / 'ps1_league_winner_best_model.joblib'
joblib.dump(best_model, model_filename)

metadata = {
    'problem_statement': 'PS1: League Winner Prediction',
    'model_type': best_model_name,
    'pipeline_steps': [step[0] for step in best_model.steps],
    'performance_metrics': {
        'accuracy': best_model_data['accuracy'],
        'f1_macro': best_model_data['f1_macro'],
        'roc_auc': best_model_data['roc_auc']
    },
    'features': feature_cols,
    'target': TARGET_COL,
    'temporal_split': {
        'train_seasons': train_seasons.tolist(),
        'test_season': int(test_season)
    },
    'training_timestamp': datetime.now().isoformat()
}

metadata_filename = MODELS_DIR / 'ps1_league_winner_metadata.json'
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=4)

print(f"Model saved to: {model_filename}")
print(f"Metadata saved to: {metadata_filename}")

KeyError: 'Test_F1'

## 6. Model Comparison